<a href="https://colab.research.google.com/github/rsmxingu/classificacao/blob/master/UNSUPERVISED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinando modelos e selecionando grupos

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [0]:
# Lendo CSV RGB
dataFrame = pd.read_csv("DataFrameSat.csv", sep= ",")

# Separando instancias das classes
instancias = dataFrame.iloc[:,0:12].values
# Separando Classes
classes = dataFrame.iloc[:,12:].values

# Calculando incidência de classe 0 ou 1 nos grupos
def calcularIncidencia(grupo,classes_, labelK):
    dados = {"qtd0":0,"qtd1":0,"percente1":0,"percente0":0}
    for x in range (len(labelK)):
        if(labelK[x]==grupo):
            if(classes_[x][0]==1):
                dados["qtd1"]+=1
            elif(classes_[x][0]==0):
                dados["qtd0"]+=1
    total = dados["qtd1"]+dados["qtd0"]
    dados["percente1"] = percente(dados["qtd1"], total)
    dados["percente0"] = percente(dados["qtd0"], total)
    return dados

def percente(valor, total):
    if total==0 :
        return 0
    return (valor*100)/total

# Aplicando K-Means

In [0]:
ini = 400; fim = 2001; passo = 100; itera = 10
for i in range(1,itera+1):
    x_train, x_test, y_train, y_test = train_test_split(instancias, classes, test_size=0.1, random_state=42)
    totalClasse1 = 0; totalClasse0 = 0;
    for x in range(len(y_test)):
        if(y_test[x]==1):
            totalClasse1+=1
        else:
            totalClasse0+=1
    resultados = []; Clusters = []
    for var in range(ini, fim, passo):
        Clusters.append(var)
        from sklearn.cluster import KMeans
        c=var
        kmeans = KMeans(n_clusters = c, init = 'random')
        kmeans.fit(x_train)

        # Gravando o resultado de todos os grupos
        gruposGeralK=[]
        for x in range(c):
            gruposGeralK.insert(x,calcularIncidencia(x,y_train,kmeans.labels_))

        # Armazenando grupos com incidência de classe 1 acima de 70% K
        grupos1K=[]
        for x in range(len(gruposGeralK)):
            if(gruposGeralK[x]['percente1']>=60):
                grupos1K.append(x)

        # Armazenando grupos com incidência de classe 0 acima de 70% K
        grupos0K=[]
        for x in range(len(gruposGeralK)):
            if(gruposGeralK[x]['percente0']>=60):
                grupos0K.append(x)

        # Testando treino
        testeK=kmeans.predict(x_test)

        # Verificando Total de acertos classe 1 K
        totalAcertos1K = 0;
        for x in range(c):
            valor =calcularIncidencia(x,y_test,testeK)
            if grupos1K.count(x)>0 :
                totalAcertos1K += valor['qtd1']

        # Verificando Total de acertos classe 0
        totalAcertos0K = 0;
        for x in range(c):
            valor =calcularIncidencia(x,y_test,testeK)
            if grupos0K.count(x)>0 :
                totalAcertos0K += valor['qtd0']
        
        resultados.append([round((totalAcertos0K*100)/totalClasse0,2),
                           round((totalAcertos1K*100)/totalClasse1,2)])
    
    percent = pd.DataFrame({'0':[resultados[x][0] for x in range(len(resultados))],'1':[resultados[x][0] for x in range(len(resultados))]})
    percent = pd.DataFrame(percent.values, index = Clusters, columns = pd.MultiIndex.from_product([['Iteração_'+str(i)], ['0', '1']]))

    if i == 1:
        KMeans_Clusters = percent
    else:
        KMeans_Clusters = pd.concat([KMeans_Clusters, percent], axis=1, join_axes=[KMeans_Clusters.index])
KMeans_Clusters.to_csv('KMeans_Clu'+str(ini)+'a'+str(fim)+'_passo'+str(passo)+'.csv')                        

# Aplicando AgglomerativeClustering

In [0]:
ini = 400; fim = 801; passo = 20; itera = 10
for i in range(1,itera+1):
    x_train, x_test, y_train, y_test = train_test_split(instancias, classes)
    totalClasse1 = 0; totalClasse0 = 0;
    for x in range(len(y_test)):
        if(y_test[x]==1):
            totalClasse1+=1
        else:
            totalClasse0+=1
    resultados = []; Clusters = []
    for var in range(ini, fim, passo):
        Clusters.append(var)
        from sklearn.cluster import AgglomerativeClustering
        c=var
        clustering = AgglomerativeClustering(n_clusters = c).fit(x_train)

        # Gravando o resultado de todos os grupos AC
        gruposGeralAC=[]
        for x in range(c):
            gruposGeralAC.insert(x,calcularIncidencia(x,y_train,clustering.labels_))

        # Agramazecando grupos com insidencia de classe 1 acima de 70% AC
        grupos1AC=[]
        for x in range(len(gruposGeralAC)):
            if(gruposGeralAC[x]['percente1']>=60):
                grupos1AC.append(x)

        # Agramazecando grupos com insidencia de classe 0 acima de 70%
        grupos0AC=[]
        for x in range(len(gruposGeralAC)):
            if(gruposGeralAC[x]['percente0']>=60):
                grupos0AC.append(x)
        
        # Testando treino
        testeAC=clustering.fit_predict(x_test)

        # Verificando Total de acertos classe 1
        totalAcertos1AC = 0;
        for x in range(c):
            valor =calcularIncidencia(x,y_test,testeAC)
            if grupos1AC.count(x)>0 :
                totalAcertos1AC += valor['qtd1']
        
        # Verificando Total de acertos classe 0
        totalAcertos0AC = 0;
        for x in range(c):
            valor =calcularIncidencia(x,y_test,testeAC)
            if grupos0AC.count(x)>0 :
                totalAcertos0AC += valor['qtd0']
        
        resultados.append([round((totalAcertos1AC*100)/totalClasse1,2),
                           round((totalAcertos0AC*100)/totalClasse0,2)])

    percent = pd.DataFrame({'0':[resultados[x][0] for x in range(len(resultados))],'1':[resultados[x][0] for x in range(len(resultados))]})
    percent = pd.DataFrame(percent.values, index = Clusters, columns = pd.MultiIndex.from_product([['Iteração_'+str(i)], ['0', '1']]))

    if i == 1:
        AgC_Clusters = percent
    else:
        AgC_Clusters = pd.concat([AgC_Clusters, percent], axis=1, join_axes=[AgC_Clusters.index])
AgC_Clusters.to_csv('AgC_Clu'+str(ini)+'a'+str(fim)+'_passo'+str(passo)+'.csv')        

# Aplicando Birch

In [0]:
ini = 1300; fim = 1901; passo = 50; itera = 10
for i in range(1,itera+1):
    x_train, x_test, y_train, y_test = train_test_split(instancias, classes, test_size=0.1, random_state=42)
    totalClasse1 = 0; totalClasse0 = 0;
    for x in range(len(y_test)):
        if(y_test[x]==1):
            totalClasse1+=1
        else:
            totalClasse0+=1
    resultados = []; Clusters = []
    for var in range(ini, fim, passo):
        Clusters.append(var)
        from sklearn.cluster import Birch
        c=var
        brc = Birch(branching_factor=50, n_clusters=c, threshold=0.5,
                    compute_labels=True)
        brc.fit(x_train)

        # Gravando o resultado de todos os grupos BR
        gruposGeralBR=[]
        for x in range(c):
            gruposGeralBR.insert(x,calcularIncidencia(x,y_train,brc.labels_))

        # Agramazecando grupos com insidencia de classe 1 acima de 70% BR
        grupos1BR=[]
        for x in range(len(gruposGeralBR)):
            if(gruposGeralBR[x]['percente1']>=70):
                grupos1BR.append(x)

        # Armazenando grupos com insidencia de classe 0 acima de 70% BR
        grupos0BR=[]
        for x in range(len(gruposGeralBR)):
            if(gruposGeralBR[x]['percente0']>=70):
                grupos0BR.append(x)
        
        # Testando treino
        testeBR=brc.predict(x_test)

        # Verificando Total de acertos classe 1
        totalAcertos1BR = 0;
        for x in range(c):
            valor =calcularIncidencia(x,y_test,testeBR)
            if grupos1BR.count(x)>0 :
                totalAcertos1BR += valor['qtd1']

        # Verificando Total de acertos classe 0
        totalAcertos0BR = 0;
        for x in range(c):
            valor =calcularIncidencia(x,y_test,testeBR)
            if grupos0BR.count(x)>0 :
                totalAcertos0BR += valor['qtd0']

        resultados.append([round((totalAcertos1BR*100)/totalClasse1,2),
                           round((totalAcertos0BR*100)/totalClasse0,2)])
    
    percent = pd.DataFrame({'0':[resultados[x][0] for x in range(len(resultados))],'1':[resultados[x][0] for x in range(len(resultados))]})
    percent = pd.DataFrame(percent.values, index = Clusters, columns = pd.MultiIndex.from_product([['Iteração_'+str(i)], ['0', '1']]))

    if i == 1:
        Birch_Clusters = percent
    else:
        Birch_Clusters = pd.concat([Birch_Clusters, percent], axis=1, join_axes=[Birch_Clusters.index])
Birch_Clusters.to_csv('Birch_Clu'+str(ini)+'a'+str(fim)+'_passo'+str(passo)+'.csv')                

In [0]:
KMeans_Clusters

Iteração_1        Iteração_2        Iteração_3        Iteração_4         \
              0      1          0      1          0      1          0      1   
400       66.05  66.05      65.43  65.43      67.28  67.28      67.28  67.28   
500       68.52  68.52      67.90  67.90      64.81  64.81      70.37  70.37   
600       67.28  67.28      64.81  64.81      68.52  68.52      70.37  70.37   
700       70.37  70.37      70.37  70.37      68.52  68.52      70.99  70.99   
800       69.75  69.75      74.07  74.07      72.22  72.22      67.28  67.28   
900       74.07  74.07      70.37  70.37      73.46  73.46      70.37  70.37   
1000      72.84  72.84      73.46  73.46      70.37  70.37      72.84  72.84   
1100      75.31  75.31      77.78  77.78      74.69  74.69      72.84  72.84   
1200      81.48  81.48      78.40  78.40      77.78  77.78      77.16  77.16   
1300      80.86  80.86      81.48  81.48      77.16  77.16      81.48  81.48   
1400      82.72  82.72      80.25  80.25      84.57  84.57      84.57  84.57   
1500      79.63  79.63      82.10  82.10      82.72  82.72      86.42  86.42   
1600      85.80  85.80      87.04  87.04      85.19  85.19      87.04  87.04   
1700      88.89  88.89      91.98  91.98      93.21  93.21      88.89  88.89   
1800      95.68  95.68      93.83  93.83      95.68  95.68      96.30  96.30   
1900      95.68  95.68      96.30  96.30      96.91  96.91      96.30  96.30   
2000      95.68  95.68      94.44  94.44      94.44  94.44      95.68  95.68   

     Iteração_5        Iteração_6        Iteração_7        Iteração_8         \
              0      1          0      1          0      1          0      1   
400       66.05  66.05      64.20  64.20      72.22  72.22      69.14  69.14   
500       61.73  61.73      62.35  62.35      63.58  63.58      63.58  63.58   
600       69.14  69.14      61.73  61.73      70.37  70.37      65.43  65.43   
700       66.05  66.05      69.14  69.14      69.75  69.75      68.52  68.52   
800       70.37  70.37      70.37  70.37      69.14  69.14      71.60  71.60   
900       70.37  70.37      68.52  68.52      71.60  71.60      74.69  74.69   
1000      73.46  73.46      75.93  75.93      77.16  77.16      75.31  75.31   
1100      74.69  74.69      82.10  82.10      76.54  76.54      76.54  76.54   
1200      75.93  75.93      77.16  77.16      79.01  79.01      79.63  79.63   
1300      79.63  79.63      79.63  79.63      79.63  79.63      79.63  79.63   
1400      81.48  81.48      85.19  85.19      82.72  82.72      79.01  79.01   
1500      83.95  83.95      85.80  85.80      84.57  84.57      84.57  84.57   
1600      87.65  87.65      87.65  87.65      87.65  87.65      86.42  86.42   
1700      90.74  90.74      90.12  90.12      91.36  91.36      91.98  91.98   
1800      96.30  96.30      95.68  95.68      94.44  94.44      94.44  94.44   
1900      97.53  97.53      95.06  95.06      95.68  95.68      94.44  94.44   
2000      95.68  95.68      94.44  94.44      93.83  93.83      95.68  95.68   

     Iteração_9        Iteração_10         
              0      1           0      1  
400       64.81  64.81       62.96  62.96  
500       62.96  62.96       68.52  68.52  
600       67.90  67.90       70.99  70.99  
700       65.43  65.43       69.75  69.75  
800       72.84  72.84       68.52  68.52  
900       72.84  72.84       70.99  70.99  
1000      67.90  67.90       77.16  77.16  
1100      77.16  77.16       75.31  75.31  
1200      74.69  74.69       76.54  76.54  
1300      78.40  78.40       79.63  79.63  
1400      82.10  82.10       75.93  75.93  
1500      83.95  83.95       79.01  79.01  
1600      84.57  84.57       87.04  87.04  
1700      93.21  93.21       93.83  93.83  
1800      95.06  95.06       95.06  95.06  
1900      95.06  95.06       95.06  95.06  
2000      96.30  96.30       96.91  96.91

In [0]:
AgC_Clusters

Iteração_1        Iteração_2        Iteração_3        Iteração_4         \
             0      1          0      1          0      1          0      1   
400      61.06  61.06      59.21  59.21      57.61  57.61      57.60  57.60   
420      58.89  58.89      62.41  62.41      58.35  58.35      59.31  59.31   
440      59.13  59.13      61.43  61.43      58.35  58.35      56.62  56.62   
460      60.34  60.34      62.65  62.65      60.60  60.60      56.13  56.13   
480      56.01  56.01      58.97  58.97      58.10  58.10      59.56  59.56   
500      58.65  58.65      59.21  59.21      56.86  56.86      60.29  60.29   
520      57.69  57.69      59.71  59.71      56.86  56.86      61.52  61.52   
540      57.21  57.21      59.21  59.21      56.11  56.11      59.31  59.31   
560      58.65  58.65      58.48  58.48      55.86  55.86      59.31  59.31   
580      59.62  59.62      56.51  56.51      56.11  56.11      57.60  57.60   
600      58.65  58.65      57.49  57.49      57.86  57.86      53.92  53.92   
620      57.21  57.21      57.00  57.00      58.60  58.60      53.43  53.43   
640      56.25  56.25      58.97  58.97      56.11  56.11      54.41  54.41   
660      58.89  58.89      59.95  59.95      55.11  55.11      53.19  53.19   
680      55.77  55.77      57.00  57.00      54.86  54.86      53.43  53.43   
700      57.21  57.21      59.46  59.46      57.36  57.36      53.92  53.92   
720      55.77  55.77      56.51  56.51      55.36  55.36      54.17  54.17   
740      55.77  55.77      56.27  56.27      54.36  54.36      55.15  55.15   
760      56.73  56.73      58.23  58.23      53.37  53.37      57.84  57.84   
780      56.73  56.73      56.27  56.27      53.62  53.62      57.60  57.60   
800      56.97  56.97      55.04  55.04      52.87  52.87      58.33  58.33   

    Iteração_5        Iteração_6        Iteração_7        Iteração_8         \
             0      1          0      1          0      1          0      1   
400      58.37  58.37      58.84  58.84      59.35  59.35      58.70  58.70   
420      58.14  58.14      59.85  59.85      59.35  59.35      61.59  61.59   
440      57.91  57.91      55.05  55.05      61.85  61.85      59.66  59.66   
460      60.23  60.23      57.07  57.07      62.84  62.84      62.08  62.08   
480      59.53  59.53      58.08  58.08      62.59  62.59      61.35  61.35   
500      59.53  59.53      57.83  57.83      63.59  63.59      60.87  60.87   
520      57.91  57.91      59.09  59.09      62.34  62.34      59.18  59.18   
540      59.77  59.77      59.85  59.85      62.34  62.34      58.94  58.94   
560      59.30  59.30      59.34  59.34      61.10  61.10      58.21  58.21   
580      58.60  58.60      60.10  60.10      61.60  61.60      58.70  58.70   
600      56.74  56.74      61.11  61.11      60.60  60.60      56.76  56.76   
620      55.81  55.81      62.37  62.37      58.85  58.85      53.86  53.86   
640      56.28  56.28      59.34  59.34      58.10  58.10      53.86  53.86   
660      57.44  57.44      60.61  60.61      56.61  56.61      53.62  53.62   
680      55.58  55.58      56.06  56.06      54.11  54.11      55.31  55.31   
700      56.05  56.05      56.31  56.31      57.11  57.11      53.62  53.62   
720      57.44  57.44      54.04  54.04      54.36  54.36      52.90  52.90   
740      58.60  58.60      55.56  55.56      54.61  54.61      49.28  49.28   
760      58.84  58.84      57.07  57.07      56.86  56.86      54.11  54.11   
780      57.67  57.67      56.57  56.57      55.36  55.36      54.59  54.59   
800      58.37  58.37      55.56  55.56      57.86  57.86      56.28  56.28   

    Iteração_9        Iteração_10         
             0      1           0      1  
400      57.87  57.87       57.21  57.21  
420      57.14  57.14       59.27  59.27  
440      58.35  58.35       59.04  59.04  
460      59.08  59.08       59.50  59.50  
480      56.17  56.17       59.04  59.04  
500      59.56  59.56       59.04  59.04  
520      61.02  61.02   

In [0]:
Birch_Clusters

Iteração_1        Iteração_2        Iteração_3        Iteração_4         \
              0      1          0      1          0      1          0      1   
1300      89.22  89.22      89.22  89.22      89.22  89.22      89.22  89.22   
1350      89.82  89.82      89.82  89.82      89.82  89.82      89.82  89.82   
1400      91.62  91.62      91.62  91.62      91.62  91.62      91.62  91.62   
1450      91.62  91.62      91.62  91.62      91.62  91.62      91.62  91.62   
1500      92.81  92.81      92.81  92.81      92.81  92.81      92.81  92.81   
1550      93.41  93.41      93.41  93.41      93.41  93.41      93.41  93.41   
1600      95.21  95.21      95.21  95.21      95.21  95.21      95.21  95.21   
1650      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1700      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1750      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1800      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1850      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1900      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   

     Iteração_5        Iteração_6        Iteração_7        Iteração_8         \
              0      1          0      1          0      1          0      1   
1300      89.22  89.22      89.22  89.22      89.22  89.22      89.22  89.22   
1350      89.82  89.82      89.82  89.82      89.82  89.82      89.82  89.82   
1400      91.62  91.62      91.62  91.62      91.62  91.62      91.62  91.62   
1450      91.62  91.62      91.62  91.62      91.62  91.62      91.62  91.62   
1500      92.81  92.81      92.81  92.81      92.81  92.81      92.81  92.81   
1550      93.41  93.41      93.41  93.41      93.41  93.41      93.41  93.41   
1600      95.21  95.21      95.21  95.21      95.21  95.21      95.21  95.21   
1650      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1700      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1750      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1800      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1850      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   
1900      96.41  96.41      96.41  96.41      96.41  96.41      96.41  96.41   

     Iteração_9        Iteração_10         
              0      1           0      1  
1300      89.22  89.22       89.22  89.22  
1350      89.82  89.82       89.82  89.82  
1400      91.62  91.62       91.62  91.62  
1450      91.62  91.62       91.62  91.62  
1500      92.81  92.81       92.81  92.81  
1550      93.41  93.41       93.41  93.41  
1600      95.21  95.21       95.21  95.21  
1650      96.41  96.41       96.41  96.41  
1700      96.41  96.41       96.41  96.41  
1750      96.41  96.41       96.41  96.41  
1800      96.41  96.41       96.41  96.41  
1850      96.41  96.41       96.41  96.41  
1900      96.41  96.41       96.41  96.41

# FIM